In [205]:
import lazypredict
import pandas as pd
from sklearn.preprocessing import RobustScaler
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, chi2, f_classif

In [206]:
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
df_X_last = pd.read_csv("data/X_train_features.csv", index_col=0)
df_X_richard = pd.read_csv("data/template_features.csv", index_col=0)
df_X_tim = pd.read_csv("full_waveform_features.csv", index_col=0)
df_X_pyHRV = pd.read_csv("data/pyHRV_features.csv", index_col=0)
df_X_hrv_analysis = pd.read_csv("data/hrv-analysis_features.csv")
df_X = pd.concat((df_X_tim,df_X_richard),axis=1)
df_X = pd.concat((df_X, df_X_pyHRV),axis=1)
df_X = pd.concat((df_X, df_X_hrv_analysis),axis=1)
# df_X = pd.concat((df_X, df_X_last),axis=1)
df_Y = pd.read_csv("data/Y_train.csv", index_col="id")

Remove highly correlated features

In [207]:
def rm_corr(X):
    corr_matrix = X.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    # Find features with correlation greater than 0.9
    to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]
    print("Removed columns: ", len(to_drop))
    # Drop features 
    X.drop(to_drop, axis=1, inplace=True)
    #X_test.drop(to_drop, axis=1, inplace=True)
    
    return X

In [208]:
df_X = rm_corr(df_X)

Removed columns:  45


In [209]:
df_X=(df_X-df_X.mean())/df_X.std()

Normalize

In [210]:
df_X.replace([np.inf, -np.inf], np.nan, inplace=True)
transformer = RobustScaler()
X = transformer.fit_transform(df_X)

In [211]:
X.shape

(5117, 79)

Impute missing values

In [212]:
imp_median = SimpleImputer(missing_values=np.nan, strategy='median')
X = imp_median.fit_transform(X)

In [214]:
X_train, X_test, y_train, y_test = train_test_split(X, df_Y,stratify=df_Y,test_size=.2,random_state=0)
clf = LazyClassifier(verbose=0, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:54<00:00,  1.87s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
XGBClassifier,0.84,0.74,None,0.84,7.93
LGBMClassifier,0.84,0.73,None,0.83,2.67
LogisticRegression,0.80,0.68,None,0.79,0.26
SGDClassifier,0.77,0.68,None,0.76,0.37
LinearDiscriminantAnalysis,0.77,0.67,None,0.76,0.16
PassiveAggressiveClassifier,0.74,0.67,None,0.73,0.13
RandomForestClassifier,0.82,0.67,None,0.81,4.25
NearestCentroid,0.67,0.66,None,0.67,0.05
GaussianNB,0.69,0.65,None,0.69,0.05
